In [1]:
# Import key packages
import numpy as np
from pyscf import gto, scf, fci
from pyscf.scf.chkfile import dump_scf
import pyci

In [2]:
def RanGo(M):
    h2 = np.zeros((2*M, 2*M, 2*M, 2*M))
    
    # build hamiltonian, i and k are uparrow, <ij|kl>
    # a b a b
    for i in range(2*M):
        for j in range(M,2*M):
            for k in range(2*M):
                for l in range(M,2*M):
                    delta_ij = 1 if i == j+M or i+M == j else 0
                    delta_kl = 1 if k == l+M or k+M == l else 0
                    delta_il = 1 if i == k else 0
                    h2[i, j, k, l] = delta_ij * delta_kl
    # b a b a 
    for i in range(M,2*M):
        for j in range(M):
            for k in range(M,2*M):
                for l in range(M):
                    delta_ij = 1 if i == j+M or i+M == j else 0
                    delta_kl = 1 if k == l+M or k+M == l else 0
                    delta_il = 1 if i == k else 0
                    h2[i, j, k, l] = delta_ij * delta_kl

    return h2

In [3]:
def Vee(eri,dm2):
    return .5*np.einsum('pqrs,pqrs', eri[0], dm2[0])\
    +.5*np.einsum('pqrs,pqrs', eri[2], dm2[2])\
    +np.einsum('pqrs,pqrs', eri[1], dm2[1])

In [5]:
from moha.hamiltonians import HamHub, HamRG

g = 1.0

M = 2
nelec = 2
adjacency = np.ones((M,M))
Richardson = HamRG(mu = np.array(range(M)), J_eq = g, adjacency)

h0_ric = Richardson.generate_zero_body_integral()
h1_ric = Richardson.generate_one_body_integral(basis='spinorbital basis',dense=True)


h2_ric = Richardson.generate_two_body_integral(sym=4,basis='spinorbital basis', dense=True)
# h2_ric = RanGo(M)*g


h2_ric_ch = np.transpose(h2_ric, (0, 2, 1, 3))

h2_ric_aa = h2_ric_ch[0:M,0:M,0:M,0:M] 
h2_ric_ab = h2_ric_ch[0:M,0:M,M:2*M,M:2*M]
h2_ric_bb = h2_ric_ch[M:2*M,M:2*M,M:2*M,M:2*M]

h1_aa = h1_ric[0:M,0:M]
h1_bb = h1_ric[M:2*M,M:2*M]

h2_ric_pyscf = (h2_ric_aa, h2_ric_ab, h2_ric_bb)
h1_ric_pyscf = (h1_aa, h1_bb)

# print(h2_ric.shape)

# print(h0_ric)
# print(h1_ric)



TypeError: HamRG.__init__() got an unexpected keyword argument 'J_ax'

In [ ]:
# np.where(h2_ric != 0)


In [ ]:
cisolver = fci.direct_uhf.FCI()
e, c = cisolver.kernel(h1_ric_pyscf, h2_ric_pyscf, M, (nelec//2, nelec//2), ecore=h0_ric)
dm1, dm2  = cisolver.make_rdm12s(c, M, (nelec//2, nelec//2))


In [ ]:
# print(len(dm2), len(h2_ric_pyscf), len(dm1))
# print(h1_ric[0])

In [ ]:
print(e, h0_ric, Vee(h2_ric_pyscf, dm2), e-2*np.trace(h1_ric_pyscf[0]@dm1[0])-h0_ric)